In [2]:
import aif360
from aif360.datasets import AdultDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing.reweighing import Reweighing
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

import numpy as np
import pandas as pd
START_BOLD = '\033[1m'
END_BOLD = '\033[0m'

In [3]:
column_names = ['age', 'workclass', 'fnlwgt', 'education',
    'education-num', 'marital-status', 'occupation', 'relationship',
    'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income-per-year']


dataset_path = Path(aif360.__file__).parent / 'data' / 'raw' / 'adult' / 'adult.data'
original_df = pd.read_csv(dataset_path, names=column_names)

In [4]:
original_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income-per-year
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


# TASK 1

## Preprocess

In [5]:
# Using as base the aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions.load_preproc_data_adult

def load_preproc_data_adult(protected_attributes=None):
    min_privileged_age = 35
    max_privileged_age = 55
    def custom_preprocessing(df):

        def is_in_privileged_age(x):
            if x > min_privileged_age and x < max_privileged_age:
                return 1.0
            else:
                return 0.0

        def group_edu(x):
            if x <= 5:
                return '<6'
            elif x >= 13:
                return '>12'
            else:
                return x

        def group_race(x):
            if x == "White":
                return 1.0
            else:
                return 0.0

        # Limit education range
        df['education years'] = df['education-num'].apply(lambda x: group_edu(x))
        df['education years'] = df['education years'].astype('category')

        # Rename income variable
        df['income binary'] = df['income-per-year']
        df['income binary'] = df['income binary'].replace(to_replace='>50K.', value='>50K', regex=True)
        df['income binary'] = df['income binary'].replace(to_replace='<=50K.', value='<=50K', regex=True)

        # Recode sex, race and binarize age
        df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})
        df['race'] = df['race'].apply(lambda x: group_race(x))
        df['age'] = df['age'].apply(lambda x: is_in_privileged_age(x))

        return df


    XD_features = ['age', 'education years', 'sex', 'race', 'hours-per-week']
    D_features = ['age', 'race', 'sex'] if protected_attributes is None else protected_attributes
    Y_features = ['income binary']
    X_features = list(set(XD_features)-set(D_features))
    categorical_features = ['education years']

    # privileged classes
    all_privileged_classes = {"age": [1.0],
                              "race": [1.0],
                              "sex": [1.0]}

    # protected attribute maps
    all_protected_attribute_maps = {"age": {1.0: f'Between {min_privileged_age} and {max_privileged_age}', 0.0: f'Not in between {min_privileged_age} and {max_privileged_age}'}, 
                                    "race": {1.0: 'White', 0.0: 'Non-white'}, 
                                    "sex": {1.0: 'Male', 0.0: 'Female'}}

    return AdultDataset(
        label_name=Y_features[0],
        favorable_classes=['>50K', '>50K.'],
        protected_attribute_names=D_features,
        privileged_classes=[all_privileged_classes[x] for x in D_features],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=X_features+Y_features+D_features,
        na_values=['?'],
        metadata={'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
                  'protected_attribute_maps': [all_protected_attribute_maps[x]
                                for x in D_features]},
        custom_preprocessing=custom_preprocessing)

In [6]:
dataset = load_preproc_data_adult(['age', 'sex'])

/tmp/ipykernel_215668/1665089171.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})


In [7]:
dataset.convert_to_dataframe()[0]

,age,race,sex,hours-per-week,education years=6,education years=7,education years=8,education years=9,education years=10,education years=11,education years=12,education years=<6,education years=>12,income binary
0,0.0,0.0,1.0,40.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,50.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,1.0,40.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,30.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,1.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
48838,1.0,1.0,1.0,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
48839,0.0,1.0,0.0,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
48840,0.0,1.0,1.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
train_test_valid_split = 0.7
test_valid_split = 0.5

train_ds, valid_test_ds = dataset.split([train_test_valid_split], shuffle=True)
validation_ds, test_ds = valid_test_ds.split([test_valid_split], shuffle=True)

## Classification

In [9]:
scale = StandardScaler()
X_train = scale.fit_transform(train_ds.features)
y_train = train_ds.labels.ravel()
w_train = train_ds.instance_weights.ravel()

dtmod = DecisionTreeClassifier()
dtmod.fit(X_train, y_train, sample_weight=w_train)
pos_ind = np.where(dtmod.classes_ == train_ds.favorable_label)[0][0]

In [10]:
validation_ds_pred = validation_ds.copy(deepcopy=True)
X_valid = scale.transform(validation_ds_pred.features)
validation_ds_pred.scores = dtmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

test_ds_pred = test_ds.copy(deepcopy=True)
X_test = scale.transform(test_ds_pred.features)
test_ds_pred.scores = dtmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

In [11]:
num_thresh = 99 
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = validation_ds_pred.scores > class_thresh
    validation_ds_pred.labels[fav_inds] = validation_ds_pred.favorable_label
    validation_ds_pred.labels[~fav_inds] = validation_ds_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(validation_ds,
                                             validation_ds_pred)
    
    ba_arr[idx] = (classified_metric_orig_valid.true_positive_rate()
                   +classified_metric_orig_valid.true_negative_rate()) /2

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

## Results

In [12]:
fav_inds = test_ds_pred.scores > best_class_thresh
test_ds_pred.labels[fav_inds] = test_ds_pred.favorable_label
test_ds_pred.labels[~fav_inds] = test_ds_pred.unfavorable_label

metric_test = ClassificationMetric(test_ds, test_ds_pred)

balanced_accuracy = (metric_test.true_negative_rate() + metric_test.true_positive_rate()) / 2
print(f"Balanced accuracy for {START_BOLD}classifier{END_BOLD}: {round(balanced_accuracy, 4)}")

Balanced accuracy for classifier: 0.7216


# TASK 2

In [13]:
very_privileged_groups = [{'sex': 1, 'age': 1}]
slightly_privileged_groups = [{'sex': 0, 'age': 1}, {'sex': 1, 'age': 0}]
privileged_groups = very_privileged_groups + slightly_privileged_groups
unprivileged_groups = [{'sex': 0, 'age': 0}]

def print_dataset_metrics(dataset, dataset_name):

    metric_very_privileged = BinaryLabelDatasetMetric(dataset, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=very_privileged_groups)

    metric_slightly_privileged = BinaryLabelDatasetMetric(dataset, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=slightly_privileged_groups)

    metric_privileged = BinaryLabelDatasetMetric(dataset, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)


    print(f"{START_BOLD}{dataset_name}{END_BOLD} metrics")
    print("Difference in mean outcomes between unprivileged and very privileged groups = %f" % metric_very_privileged.mean_difference())
    print("Difference in mean outcomes between unprivileged and slightly privileged groups = %f" % metric_slightly_privileged.mean_difference())
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_privileged.mean_difference())



def print_classifier_metrics(base_ds, prediction_ds, best_class_thresh, classifier_name):
    fav_inds = prediction_ds.scores > best_class_thresh
    prediction_ds.labels[fav_inds] = prediction_ds.favorable_label
    prediction_ds.labels[~fav_inds] = prediction_ds.unfavorable_label

    metric_test = ClassificationMetric(base_ds, prediction_ds,
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)

    balanced_accuracy = (metric_test.true_negative_rate() + metric_test.true_positive_rate()) / 2
    statistical_parity_difference = metric_test.statistical_parity_difference()
    disparate_impact = metric_test.disparate_impact()
    average_odds_difference = metric_test.average_odds_difference()
    equal_opportunity_difference = metric_test.equal_opportunity_difference()
    theil_index = metric_test.theil_index()

    print(f"Balanced accuracy for {START_BOLD}{classifier_name}{END_BOLD}: {round(balanced_accuracy, 4)}")
    print(f"Statistical parity difference for {START_BOLD}{classifier_name}{END_BOLD}: {round(statistical_parity_difference, 4)}")
    print(f"Disparate impact for {START_BOLD}{classifier_name}{END_BOLD}: {round(disparate_impact, 4)}")
    print(f"Average odds difference for {START_BOLD}{classifier_name}{END_BOLD}: {round(average_odds_difference, 4)}")
    print(f"Equal opportunity difference for {START_BOLD}{classifier_name}{END_BOLD}: {round(equal_opportunity_difference, 4)}")
    print(f"Theil index for {START_BOLD}{classifier_name}{END_BOLD}: {round(theil_index, 4)}")

## Fairness Metrics

In [14]:
print_dataset_metrics(train_ds, "Unfair Dataset")

Unfair Dataset metrics
Difference in mean outcomes between unprivileged and very privileged groups = -0.370784
Difference in mean outcomes between unprivileged and slightly privileged groups = -0.130969
Difference in mean outcomes between unprivileged and privileged groups = -0.217735


In [15]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=very_privileged_groups)
RW.fit(train_ds)
fair_train_ds = RW.transform(train_ds)

In [16]:
print_dataset_metrics(fair_train_ds, "Fair Dataset")

Fair Dataset metrics
Difference in mean outcomes between unprivileged and very privileged groups = 0.000000
Difference in mean outcomes between unprivileged and slightly privileged groups = 0.040390
Difference in mean outcomes between unprivileged and privileged groups = 0.025777


## Fair Classifier

In [17]:
fair_scale = StandardScaler()
fair_X_train = fair_scale.fit_transform(fair_train_ds.features)
fair_y_train = fair_train_ds.labels.ravel()
fair_w_train = fair_train_ds.instance_weights

fair_dtmod = DecisionTreeClassifier()
fair_dtmod.fit(fair_X_train, fair_y_train, sample_weight=fair_w_train)

DecisionTreeClassifier()

In [18]:
fair_test_ds_pred = test_ds.copy(deepcopy=True)
fair_X_test = fair_scale.transform(fair_test_ds_pred.features)
fair_y_test = fair_test_ds_pred.labels
fair_test_ds_pred.scores = fair_dtmod.predict_proba(fair_X_test)[:,pos_ind].reshape(-1, 1)

## Results

In [19]:
print_classifier_metrics(test_ds, test_ds_pred, best_class_thresh, "classifier")

Balanced accuracy for classifier: 0.7216
Statistical parity difference for classifier: -0.407
Disparate impact for classifier: 0.1525
Average odds difference for classifier: -0.4178
Equal opportunity difference for classifier: -0.5314
Theil index for classifier: 0.1211


In [20]:
print_classifier_metrics(test_ds, fair_test_ds_pred, best_class_thresh, "fair classifier")

Balanced accuracy for fair classifier: 0.6752
Statistical parity difference for fair classifier: -0.0753
Disparate impact for fair classifier: 0.7798
Average odds difference for fair classifier: -0.0098
Equal opportunity difference for fair classifier: -0.0219
Theil index for fair classifier: 0.1541


# Task 3